# Обзор модуля

Модуль `gmdh` содержит:
* функции предобработки данных;
* внешнии критерии;
* модели алгоритмов МГУА.

## Функции предобработки данных

В модуле представлена реализация двух функций предобработки данных:
* `time_series_transformation()` - чтобы сформировать из временного ряда массивы $X$ и $y$.

    С помощью параметра `lags` можно задать длину подпоследовательностей из временного ряда, которые образуют строки матрицы $X$.
    <br><br> 
* `split_data()` для разбиения данных на две подвыборки.

    С помощью параметра `test_size`, который принимает значения от 0 до 1, можно указать долю данных второй (проверочной) подвыборки.


(create_train_data)=
### Примеры

In [1]:
# импортируем функции предобработки данных
from gmdh import time_series_transformation, split_data

# создаём временной ряд
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  

# разбиваем временной ряд на последовательности из 4 элементов
# и сохраняем данные в матрицы X и y
X, y = time_series_transformation(data, lags=4)

## выводим матрицы
print(X,'\n')
print(y)

[[1. 2. 3. 4.]
 [2. 3. 4. 5.]
 [3. 4. 5. 6.]
 [4. 5. 6. 7.]
 [5. 6. 7. 8.]
 [6. 7. 8. 9.]] 

[ 5.  6.  7.  8.  9. 10.]


In [2]:
# разбиваем данные на обучающую (67%) и проверочную (33%) выборки
x_train, x_test, y_train, y_test = split_data(X, y, test_size=0.33)

# выводим матрицы
print(x_train,'\n')
print(x_test,'\n')
print(y_train,'\n')
print(y_test)

[[1. 2. 3. 4.]
 [2. 3. 4. 5.]
 [3. 4. 5. 6.]
 [4. 5. 6. 7.]] 

[[5. 6. 7. 8.]
 [6. 7. 8. 9.]] 

[5. 6. 7. 8.] 

[ 9. 10.]


## Внешние критерии

В модуле представлено три класса типов внешних критериев:
* `Criterion` - индивидуальный критерий;
* `ParallelCriterion` - параллельный критерий;
* `SequentialCriterion` - последовательный критерий.

Все виды индивидуальных критериев доступны через объект `CriterionType`:
* `CriterionType.REGULARITY`;
* `CriterionType.SYM_REGULARITY`;
* `CriterionType.STABILITY`;
* `CriterionType.SYM_STABILITY`;
* `CriterionType.UNBIASED_COEFFS`;
* `CriterionType.UNBIASED_OUTPUTS`;
* `CriterionType.SYM_UNBIASED_OUTPUTS`;
* `CriterionType.ABSOLUTE_NOISE_IMMUNITY`;
* `CriterionType.SYM_ABSOLUTE_NOISE_IMMUNITY`.

При создании любого из критериев можно выбрать алгоритм, который будет использоваться при решении систем линейных уравнений для вычисления оптимальных весовых коэффицентов моделей. Для этого можно воспользоваться объектом `Solver`. Доступен выбор из трёх видов алгоритмов: 
* `Solver.FAST` - самый быстрый из трёх алгоритмов;
* `Solver.ACCURATE` - самый точный из трёх алгоритмов;
* `Solver.BALANCED` - алгоритм, сбалансированный как по времени, так и по точности.

По умолчанию всегда используется сбалансированный алгоритм. При этом разница в рассчитаных значениях и скорости работы алгоритмов на небольших и средних по размеру выборках практически отсутствует.

### Пример

***
Для начала импортируем класс с перечислением индивидуальных критерии:

In [3]:
from gmdh import CriterionType

***
Для создания индивидуального критерия регулярности нужно задать значение `CriterionType.REGULARITY` параметру `сriterion_type` у класса `Criterion`:

In [4]:
from gmdh import Criterion

reg_criterion = Criterion(criterion_type=CriterionType.REGULARITY)

***
Для создания параллельного критерия, состоящего из критериев регулярности и стабильности нужно задать значение `CriterionType.REGULARITY`  паарметру `criterion_type` и значение `CriterionType.STABILITY` параметру `second_criterion_type` у класса `ParallelCriterion`. А чтобы сделать вклад критерия регулярности в итоговый результат в 3 раза больше критерия стабильности, присвоим параметру `alpha` значение 0.75 (то есть вклад первого критерия будет с коэффициентом 0.75, а вклад второго - 0.25):

In [5]:
from gmdh import ParallelCriterion

par_criterion = ParallelCriterion(criterion_type=CriterionType.REGULARITY, 
                                  second_criterion_type=CriterionType.STABILITY,
                                  alpha=0.75)

***
Для создания последовательного критерия, состоящего из симметричных критериев регулярности и стабильности нужно задать значение `CriterionType.SYM_REGULARITY`  паарметру `criterion_type` и значение `CriterionType.SYM_STABILITY` параметру `second_criterion_type` у класса `SequentialCriterion`. А чтобы использовать при обучении самый быстрый алгоритм расчёта параметров моделей, присвоим параметру `solver` значение `Solver.FAST`:

In [6]:
from gmdh import SequentialCriterion, Solver

seq_criterion = SequentialCriterion(criterion_type=CriterionType.SYM_REGULARITY, 
                                    second_criterion_type=CriterionType.SYM_STABILITY, 
                                    solver=Solver.FAST)

***
## Модели алгоритмов МГУА

В модуле `gmdh` представлено 4 класса алгоритмов МГУА:
* `Combi` - комбинаторный алгоритм МГУА;
* `Multi` - комбинаторно-селекционный алгоритм МГУА;
* `Mia` - многорядный итеративный алгоритм МГУА;
* `Ria` - релаксационный итеративный алгорим МГУА.

Для упрощения работы с этими классами, все они имеют единый список методов:
* `fit()` - для обучения алгоритма;
* `get_best_polynomial()` - для отображения полинома оптимальной модели, найденной алгоритмом и используемой при прогнозах;
* `predict()` - для вычисления прогнозов после обучения алгоритма;
* `save()` - для сохранения обученной модели в JSON-файл;
* `load()` - для загрузки предобученной модели из JSON-файла.

Рассмотрим использование данных методов на примере обучения модели `Multi`. 

Для начала импортируем соответствующий класс и создадим модель:

In [7]:
from gmdh import Multi

model = Multi()

### Метод `fit`

Метод предназначен для обучения алгоритма и позволяет задавать значения всем необходимым гиперпараметрам.

Минимально необходимый список параметров, которые нужно передать методу: обучающая выборка в виде матрицы $X$ (данные, по которым строим прогнозы) и вектора $y$ (значения целевой величины, которую предсказываем).

При желании можно также задать значения {ref}`гиперпараметров<hyperparameters>`. Более того, метод `fit` у всех алгоритмов имеет несколько дополнительных параметров, которые не влияют на *результат* обучения, но могут сделать процесс обучения более быстрым и наглядным:
* `n_jobs` - число потоков, в которых будут проводиться все вычисления в процессе обучения.

    Допускается задавать абсолютное число потоков, либо присваивать значение $-1$ для использования максимально допустимого числа потоков.
    <br>
* `verbose` - отвечает за вывод интерактивной информации о процессе обучения.

    При значении 1 выводится индикатор выполнения обучения с указанием числа рассматриваемых комбинаций на каждом ряду, а также значением качества ряда. При значении 0 вывод информации скрывается.

Чтобы оперативно узнать список доступных параметров используемого алгоритма, их описания, а также значений по умолчанию, можно выполнить команду:

In [8]:
help(model.fit)

Help on method fit in module gmdh.gmdh:

fit(X, y, criterion=<gmdh.gmdh.Criterion object at 0x0000024BFB6FFEE0>, k_best=1, test_size=0.5, p_average=1, n_jobs=1, verbose=0, limit=0) method of gmdh.gmdh.Multi instance
    Fitting the Multi model to find the best solution.
    
    Using the input training `X` and `y` data the Multi model looks for the optimal solution
    from linear functions and stops if the errors start to grow.
    The training process can be configured for specific purposes
    using hyperparameters.
    
    Parameters
    ----------
    X : array_like
        2D array containing numeric training data.
        Rows are data samples, columns are features or lags.
    y : array_like
        1D array containg target numeric values for the training data.
    criterion : gmdh.Criterion, default=gmdh.Criterion(gmdh.CriterionType.REGULARITY)
        External criterion is a function or specific combination of functions
        for solutions evaluation and choosing the best

:::{Caution}
Обратите внимание, что гиперпараметр `test_size`, используемый в методе `fit` отвечает за способ разделения *обучающей* выборки $X$, которую вы подаёте в метод, на тренировочную и тестовую части. Само разделение данных произойдёт уже внутри метода `fit` без необходимости вручную вызывать функцию `split_data()`.

При этом разделение исходной выборки на *обучающую* и *проверочную* части на этапе предобработки данных необходимо всегда выполнять самостоятельно посредством вызова функции `split_data()`

Подробнее о выборках написано в разделе {ref}`Разбиение данных на три подвыборки<subsamples>`. 
:::

<br>

Итак, обучим модель на обучающей выборке, полученной с помощью функции `split_data` в {ref}`примере<create_train_data>` предобработки данных, и зададим значения нескольких гиперпараметров:

In [9]:
model.fit(x_train, y_train, k_best=2, test_size=0.3)

После того, как метод `fit` закончит свою работу, объект `model` будет содержать уже готовую обученную модель.

### Метод `get_best_polynomial()`

Метод позволяет отобразить найденную алгоритмом во время обучения зависимость целевой величины $y$ от переменных $X$.

При использовании алгоритмов `Combi` или `Multi` вывод будет представлять собой один полином, характеризующий оптимальную модель, по которой в дальнейшем будут совершаться прогнозы.

Выведем полином нашей обученной модели:

In [10]:
model.get_best_polynomial()

'y = x1 + 4'

В случае с алгоритмами `Mia` и `Ria` вывод будет выглядить значительно сложнее. Это объясняется тем, что структура генерируемых этими алгоритмами моделей имеет рекурсивный характер, то есть в лучшем полиноме в качестве переменных будут выступать другие полиномы, каждый из которых может тоже состоять из полиномов и так далее. Поэтому использовать вывод лучшего полинома при использовании данных алгоритмов целесообразно при небольшом числе рядов, когда уровней вложенности полиномов немного. 

Обучим для примера на тех же данных модель `Mia` и выведем лучший полином:

In [11]:
from gmdh import Mia

mia_model = Mia()
mia_model.fit(x_train, y_train, k_best=5, p_average=3)
print(mia_model.get_best_polynomial())

f1_1 = - 0.4833*x1*x4 + 0.4333*x4^2
f1_2 = - 1.5294*x2^2 + 1.2353*x3^2

y = f1_1 + 1.82381e-16*f1_1^2


### Метод `predict()`

Метод используется для совершения прогнозов на уже обученной модели. Для этого используются новые данные, которые модель не видела во время обучения. В нашем случае можно воспользоваться проверочной выборкой, полученной в {ref}`примере<create_train_data>` предобработки данных.

Метод может работать в двух режимах:
* одновременный прогноз для всех строк переданной матрицы $X$.

    К этому режиму относятся также и краткосрочные прогнозы по временному ряду на 1 шаг вперёд. В таком случае каждая строка матрицы $X$ представляет собой подпоследовательность временного ряда и метод `predict` вернёт предсказания одного следующего числа для каждой подпоследовательности.
    <br>
* долгосрочный прогноз по одной подпоследовательности временного ряда.

    В этом случае количество шагов, на которые требуется последовательно предсказать значения, можно задать параметром `lags`.

Пример использования первого режима:

In [12]:
# выведем проверочную выборку
print('x_test:\n', x_test, '\n')

# строим прогноз
y_pred = model.predict(x_test)

# сравниваем реальные значение с прогнозом
print('y_test: ', y_test)
print('y_pred: ', y_pred)

x_test:
 [[5. 6. 7. 8.]
 [6. 7. 8. 9.]] 

y_test:  [ 9. 10.]
y_pred:  [ 9. 10.]


Для примера использоваиня второго режима подадим в метод только первую строку матрицы `x_test` и попросим предсказать сразу 5 следующих значений временного ряда:

In [13]:
# выводим начало подпоследовательности временного ряда
print('x_test[0]:', x_test[0])

# строим прогноз 5 значений
y_pred = model.predict(x_test[0], lags=5)

# выводим прогноз
print('y_pred:', y_pred)

x_test[0]: [5. 6. 7. 8.]
y_pred: [ 9. 10. 11. 12. 13.]


### Метод `save()`

Метод позволяет сохранить обученную модель в JSON-файл.

Для этого достаточно указать лишь название файла:

In [14]:
model.save('multi_model.json')

### Метод `load()`

Метод позволяет использовать готовые предобученные модели, которые ранее были сохранены в JSON-файл. Так как при больших выборках процесс обучения может занимать много времени, то возможность обучить алгоритм всего один раз, а затем подгружать модель в любой скрипт и сразу строить прогнозы позволяет экономить время.

Для загрузки обученной модели необходимо создать объект того же алгоритма МГУА, что и модель в файле и затем указать путь к файлу:

In [15]:
# создаём объект алгоритма Multi
pretrained_model = Multi()

# загружаем предобученную модель
pretrained_model.load('multi_model.json')

# выводим полином, чтобы убедиться, что модель загрузилась
pretrained_model.get_best_polynomial()

'y = x1 + 4'